In [ ]:
# ising3d.py
import numpy as np


class Ising3D:
    """
    3D Ising model on an LxLxL cubic lattice with periodic boundary conditions.
    Spins S[i,j,k] = ±1.

    Parameters
    ----------
    L : int
        Linear system size (N = L^3 spins).
    T : float
        Temperature (k_B = 1, J = 1 by default).
    J : float
        Coupling constant (ferromagnetic if J > 0).
    h : float
        External magnetic field (default 0).
    rng : np.random.Generator or None
        Random number generator. If None, a new default_rng() is created.
    """

    def __init__(self, L, T, J=1.0, h=0.0, rng=None):
        self.L = int(L)
        self.N = self.L ** 3
        self.J = float(J)
        self.h = float(h)
        self.set_temperature(T)

        self.rng = rng if rng is not None else np.random.default_rng()

        # Spins as int8 to save memory, ±1
        self.spins = np.ones((self.L, self.L, self.L), dtype=np.int8)

    # ----------------------------
    # Basic configuration methods
    # ----------------------------
    def set_temperature(self, T):
        self.T = float(T)
        if self.T <= 0:
            raise ValueError("Temperature T must be positive.")
        self.beta = 1.0 / self.T

    def randomize_spins(self):
        """Random initial condition: spins = ±1 with equal probability."""
        self.spins = self.rng.choice([-1, 1], size=(self.L, self.L, self.L)).astype(np.int8)

    def set_all_up(self):
        """Ordered initial condition: all spins +1."""
        self.spins.fill(1)

    # ----------------------------
    # Energy and magnetisation
    # ----------------------------
    def magnetisation(self):
        """Total magnetisation M and magnetisation per spin m."""
        M = np.sum(self.spins, dtype=np.int64)
        m = M / self.N
        return M, m

    def _neighbor_sum(self, i, j, k):
        """Sum of nearest neighbours of spin at (i,j,k) with periodic boundaries."""
        L = self.L
        s = self.spins
        return (
            s[(i + 1) % L, j, k] +
            s[(i - 1) % L, j, k] +
            s[i, (j + 1) % L, k] +
            s[i, (j - 1) % L, k] +
            s[i, j, (k + 1) % L] +
            s[i, j, (k - 1) % L]
        )

    def local_energy(self, i, j, k):
        """
        Local energy contribution of spin at (i,j,k) including field:
        E_i = -J * S_i * sum_{nn} S_j - h * S_i
        """
        Si = self.spins[i, j, k]
        nn_sum = self._neighbor_sum(i, j, k)
        return -self.J * Si * nn_sum - self.h * Si

    def total_energy(self):
        """
        Total energy of the system.
        For efficiency, compute via nearest-neighbour bonds once.
        """
        s = self.spins
        J = self.J
        h = self.h

        # Periodic neighbour sums in +x, +y, +z directions to avoid double counting
        E_bonds = 0.0
        E_bonds += np.sum(s * np.roll(s, shift=-1, axis=0))  # x-direction
        E_bonds += np.sum(s * np.roll(s, shift=-1, axis=1))  # y-direction
        E_bonds += np.sum(s * np.roll(s, shift=-1, axis=2))  # z-direction

        E_bonds *= -J  # -J sum_{<ij>} S_i S_j

        # Field term: -h sum_i S_i
        E_field = -h * np.sum(s)

        return float(E_bonds + E_field)

    # ----------------------------
    # Metropolis single-spin update
    # ----------------------------
    def metropolis_sweep(self):
        """
        One Metropolis sweep = N attempted single-spin flips.

        Returns
        -------
        accept_rate : float
            Fraction of accepted spin flips during this sweep.
        """
        L = self.L
        s = self.spins
        beta = self.beta
        J = self.J
        h = self.h

        accepted = 0

        for _ in range(self.N):
            # Pick random site
            i = self.rng.integers(0, L)
            j = self.rng.integers(0, L)
            k = self.rng.integers(0, L)

            Si = s[i, j, k]
            nn_sum = self._neighbor_sum(i, j, k)

            # ΔE for flipping Si -> -Si:
            # before: E_i = -J Si nn_sum - h Si
            # after:  E_i' = -J (-Si) nn_sum - h (-Si) = J Si nn_sum + h Si
            # ΔE = E_i' - E_i = 2 J Si nn_sum + 2 h Si
            dE = 2.0 * Si * (J * nn_sum + h)

            if dE <= 0.0:
                s[i, j, k] = -Si
                accepted += 1
            else:
                if self.rng.random() < np.exp(-beta * dE):
                    s[i, j, k] = -Si
                    accepted += 1

        return accepted / self.N

    # ----------------------------
    # Wolff cluster update
    # ----------------------------
    def wolff_cluster_step(self):
        """
        Perform one Wolff cluster flip and return the cluster size.

        We build a cluster of like spins starting from a random seed and
        add neighbours with probability p_add = 1 - exp(-2 β J).
        """
        L = self.L
        s = self.spins
        beta = self.beta
        J = self.J

        # Bond-addition probability
        p_add = 1.0 - np.exp(-2.0 * beta * J)

        # Choose random seed spin
        i0 = self.rng.integers(0, L)
        j0 = self.rng.integers(0, L)
        k0 = self.rng.integers(0, L)

        spin_seed = s[i0, j0, k0]

        # Boolean visited mask to avoid re-adding sites
        visited = np.zeros_like(s, dtype=bool)

        # Stack / queue for cluster growth
        stack = [(i0, j0, k0)]
        visited[i0, j0, k0] = True

        cluster_sites = []

        while stack:
            i, j, k = stack.pop()
            cluster_sites.append((i, j, k))

            # Check all 6 neighbours
            neighbours = (
                ((i + 1) % L, j, k),
                ((i - 1) % L, j, k),
                (i, (j + 1) % L, k),
                (i, (j - 1) % L, k),
                (i, j, (k + 1) % L),
                (i, j, (k - 1) % L),
            )

            for ni, nj, nk in neighbours:
                if visited[ni, nj, nk]:
                    continue
                if s[ni, nj, nk] != spin_seed:
                    continue
                # Same spin as seed; consider adding to cluster
                if self.rng.random() < p_add:
                    visited[ni, nj, nk] = True
                    stack.append((ni, nj, nk))

        # Flip entire cluster
        for (i, j, k) in cluster_sites:
            s[i, j, k] = -s[i, j, k]

        cluster_size = len(cluster_sites)
        return cluster_size

    def wolff_effective_sweep(self):
        """
        Perform Wolff cluster flips until the total number of flipped spins
        is at least N. This defines one 'effective sweep' comparable to
        one Metropolis sweep.

        Returns
        -------
        n_clusters : int
            Number of clusters flipped.
        mean_cluster_size : float
            Mean cluster size during this effective sweep.
        """
        total_flipped = 0
        cluster_sizes = []

        while total_flipped < self.N:
            size = self.wolff_cluster_step()
            cluster_sizes.append(size)
            total_flipped += size

        n_clusters = len(cluster_sizes)
        mean_size = np.mean(cluster_sizes)
        return n_clusters, float(mean_size)


In [ ]:
# run_simulation.py
import numpy as np
from ising3d import Ising3D


def run_simulation(
    L,
    T,
    algo="metropolis",
    J=1.0,
    h=0.0,
    n_equil_sweeps=2000,
    n_meas_sweeps=5000,
    meas_interval=10,
    seed=None,
):
    """
    Run a single simulation at fixed (L, T, J, h) with either Metropolis or Wolff.

    Returns
    -------
    results : dict
        Contains time series of observables and metadata:
        {
            "L": L,
            "T": T,
            "algo": algo,
            "sweeps": sweeps_array,
            "E": energies,
            "M": magnetisations,
            "m": magnetisation_per_spin,
            "accept_rate": accept_rates or None,
            "cluster_info": list of (n_clusters, mean_size) or None
        }
    """

    rng = np.random.default_rng(seed)

    model = Ising3D(L=L, T=T, J=J, h=h, rng=rng)
    # Choose initial condition; you can switch to set_all_up() if needed
    model.randomize_spins()

    energies = []
    mags = []
    mags_per_spin = []
    sweeps = []
    accept_rates = []
    cluster_stats = []

    # 1) Equilibration
    if algo.lower() == "metropolis":
        for _ in range(n_equil_sweeps):
            model.metropolis_sweep()

    elif algo.lower() == "wolff":
        for _ in range(n_equil_sweeps):
            model.wolff_effective_sweep()

    else:
        raise ValueError("algo must be 'metropolis' or 'wolff'.")

    # 2) Measurement phase
    sweep_counter = 0

    for meas_step in range(n_meas_sweeps):
        if algo.lower() == "metropolis":
            acc = model.metropolis_sweep()
            accept_rates.append(acc)
        else:
            n_clusters, mean_size = model.wolff_effective_sweep()
            cluster_stats.append((n_clusters, mean_size))

        sweep_counter += 1

        # Record measurements every meas_interval sweeps
        if (meas_step % meas_interval) == 0:
            E = model.total_energy()
            M, m = model.magnetisation()

            energies.append(E / model.N)  # energy per spin
            mags.append(M)
            mags_per_spin.append(m)
            sweeps.append(sweep_counter)

    results = {
        "L": L,
        "T": T,
        "J": J,
        "h": h,
        "algo": algo.lower(),
        "sweeps": np.array(sweeps),
        "E": np.array(energies),
        "M": np.array(mags),
        "m": np.array(mags_per_spin),
        "accept_rate": np.array(accept_rates) if algo.lower() == "metropolis" else None,
        "cluster_info": np.array(cluster_stats) if algo.lower() == "wolff" else None,
    }

    return results


if __name__ == "__main__":
    # Tiny smoke test run (this is not a real production run)
    L = 8
    T = 4.5

    res_met = run_simulation(L=L, T=T, algo="metropolis", n_equil_sweeps=500, n_meas_sweeps=500)
    res_wolff = run_simulation(L=L, T=T, algo="wolff", n_equil_sweeps=500, n_meas_sweeps=500)

    print("Metropolis: mean e =", np.mean(res_met["E"]), "mean |m| =", np.mean(np.abs(res_met["m"])))
    print("Wolff     : mean e =", np.mean(res_wolff["E"]), "mean |m| =", np.mean(np.abs(res_wolff["m"])))
